# Tutorial MultiVI

following https://docs.scvi-tools.org/en/1.0.1/tutorials/notebooks/MultiVI_tutorial.html

also intergation benchmarking dataset from https://openproblems.bio/events/2021-09_neurips/ and https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE194122

In [2]:
import numpy as np
import scanpy as sc
import scvi

/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# !wget -P data https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_unsorted_10k/pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz
# !sudo tar -C data -xzvf data/pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz 
# !gunzip -f data/filtered_feature_bc_matrix/*.gz

--2023-10-06 13:02:54--  https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_unsorted_10k/pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.0.173, 104.18.1.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375260850 (358M) [application/x-tar]
Saving to: ‘data/pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz’

pbmc_unsorted_10k_f 100%[===================>] 357.88M  16.4MB/s    in 26s     

2023-10-06 13:03:21 (14.0 MB/s) - ‘data/pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz’ saved [375260850/375260850]

tar: pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
gzip: filtered_feature_bc_matrix/*.gz: No such file or directory


In [3]:
scvi.settings.seed = 420

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

Global seed set to 420


In [4]:
# read multiomic data
adata = scvi.data.read_10x_multiome("data/filtered_feature_bc_matrix")
adata.var_names_make_unique()

/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
# We can now use the organizing method from scvi to concatenate these anndata
adata_mvi = scvi.data.organize_multiome_anndatas(adata )

/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [6]:
adata

AnnData object with n_obs × n_vars = 12012 × 148458
    obs: 'batch_id'
    var: 'ID', 'modality', 'chr', 'start', 'end'

In [18]:
adata.var["modality"]

MIR1302-2HG               Gene Expression
FAM138A                   Gene Expression
OR4F5                     Gene Expression
AL627309.1                Gene Expression
AL627309.3                Gene Expression
                               ...       
KI270713.1:21448-22354              Peaks
KI270713.1:26124-26873              Peaks
KI270713.1:29681-30482              Peaks
KI270713.1:34378-35108              Peaks
KI270713.1:36924-37836              Peaks
Name: modality, Length: 148458, dtype: object

In [16]:
print(adata.raw)

None


In [7]:
adata_mvi


AnnData object with n_obs × n_vars = 12012 × 148458
    obs: 'batch_id', 'modality'
    var: 'ID', 'modality', 'chr', 'start', 'end'

In [8]:
adata_mvi.obs

,batch_id,modality
AAACAGCCAAAGCCTC_paired,1,paired
AAACAGCCAGAATGAC_paired,1,paired
AAACAGCCAGCTACGT_paired,1,paired
AAACAGCCAGGCCTTG_paired,1,paired
AAACAGCCATAAGGAC_paired,1,paired
...,...,...
TTTGTTGGTACGCGCA_paired,1,paired
TTTGTTGGTATTTGCC_paired,1,paired
TTTGTTGGTGATTACG_paired,1,paired
TTTGTTGGTTTCAGGA_paired,1,paired


In [9]:
# filter out features
print(adata_mvi.shape)
sc.pp.filter_genes(adata_mvi, min_cells=int(adata_mvi.shape[0] * 0.01))
print(adata_mvi.shape)

(12012, 148458)


/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


(12012, 97533)


In [10]:
#scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key="modality")
scvi.model.MULTIVI.setup_anndata(adata_mvi )


I0000 00:00:1696843749.709197    6420 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [11]:
adata_mvi

AnnData object with n_obs × n_vars = 12012 × 97533
    obs: 'batch_id', 'modality', '_indices', '_scvi_batch', '_scvi_labels'
    var: 'ID', 'modality', 'chr', 'start', 'end', 'n_cells'
    uns: '_scvi_uuid', '_scvi_manager_uuid'

In [23]:
adata_mvi.uns["_scvi_uuid"]

'2ebc2850-d9dc-46b1-9657-2e9d539f6054'

In [12]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var["modality"] == "Gene Expression").sum(),
    n_regions=(adata_mvi.var["modality"] == "Peaks").sum(),
)
mvi.view_anndata_setup()

/home/jupyter/analysis/dis2p_trials/scfair_20231003/scfair_dev_20231003_env/lib/python3.10/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Anndata setup with scvi-tools version 1.0.3.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None,
│   'protein_expression_obsm_key': None,
│   'protein_names_uns_key': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │   1   │
│         n_cells          │ 12012 │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │   1   │
│          n_vars          │ 97533 │
└──────────────────────────┴───────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    ind_x     │   adata.obs['_indices']   │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                     batch State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_batch'] │     0      │          0          │
└──────────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                     batch State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_batch'] │     0      │          0          │
└──────────────────────────┴────────────┴─────────────────────┘

## Now let's use the integration benchmarking dataset

In [13]:
adata1 = scvi.data.read_h5ad("data/GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad")
adata1.var_names_make_unique()